# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file_path = "Output/weather.csv"
weather_df = pd.read_csv(file_path)
weather_df


,City,Temperature F,Max Temperature F,Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Country,Date,City Id
0,Saint-Philippe,64.26,68.00,-21.36,55.77,77,37,4.70,RE,1598812760,935215
1,San Policarpo,79.79,79.79,12.18,125.51,83,100,7.83,PH,1598812761,1688696
2,Arraial do Cabo,83.95,86.00,-22.97,-42.03,61,0,14.99,BR,1598812761,3471451
3,Manado,75.20,75.20,1.49,124.85,94,20,2.24,ID,1598812481,1636544
4,Tuatapere,49.57,49.57,-46.13,167.68,66,70,9.95,NZ,1598812762,2180815
...,...,...,...,...,...,...,...,...,...,...,...
550,Kitgum,68.25,68.25,3.28,32.89,88,37,2.42,UG,1598813052,230893
551,Kiunga,72.50,72.50,-6.12,141.29,98,94,1.34,PG,1598813052,2093846
552,Kumul,70.14,70.14,42.80,93.45,23,0,5.84,CN,1598813054,1529484
553,Mamaku,50.56,53.01,-38.10,176.08,91,79,1.01,NZ,1598813055,2187834


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [5]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Store humidity in weight
weights = weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_df =weather_df.loc[(weather_df["Max Temperature F"]< 80) & (weather_df["Humidity"]<40 ) & (weather_df["Wind Speed"]<5 ), :]
vacation_df

,City,Temperature F,Max Temperature F,Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Country,Date,City Id
56,Ntcheu,69.42,69.42,-14.82,34.64,33,0,3.74,MW,1598812778,924492
121,Shevchenkove,75.11,75.20,49.70,37.17,33,0,4.47,UA,1598812804,694137
124,Menongue,65.62,65.62,-14.66,17.69,32,6,2.37,AO,1598812805,3347353
182,San Juan,60.78,60.78,-31.54,-68.54,32,0,3.80,AR,1598812668,3837213
228,Yumen,55.74,55.74,40.28,97.20,38,0,4.27,CN,1598812865,1528998
247,Ondjiva,70.57,70.57,-17.07,15.73,25,1,4.12,AO,1598812877,3346821
358,Taloqan,65.57,65.57,36.74,69.53,36,3,2.39,AF,1598812941,1123004
362,Leh,60.57,60.57,34.17,77.58,30,0,4.29,IN,1598812944,1264976
427,Bloemfontein,39.20,39.20,-29.12,26.21,25,0,3.36,ZA,1598812980,1018725


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# set up additional columns to hold information
vacation_df['Hotel Name'] = ""
hotel_df = vacation_df
hotel_df

<ipython-input-7-3e54d2785500>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df['Hotel Name'] = ""


,City,Temperature F,Max Temperature F,Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Country,Date,City Id,Hotel Name
56,Ntcheu,69.42,69.42,-14.82,34.64,33,0,3.74,MW,1598812778,924492,
121,Shevchenkove,75.11,75.20,49.70,37.17,33,0,4.47,UA,1598812804,694137,
124,Menongue,65.62,65.62,-14.66,17.69,32,6,2.37,AO,1598812805,3347353,
182,San Juan,60.78,60.78,-31.54,-68.54,32,0,3.80,AR,1598812668,3837213,
228,Yumen,55.74,55.74,40.28,97.20,38,0,4.27,CN,1598812865,1528998,
247,Ondjiva,70.57,70.57,-17.07,15.73,25,1,4.12,AO,1598812877,3346821,
358,Taloqan,65.57,65.57,36.74,69.53,36,3,2.39,AF,1598812941,1123004,
362,Leh,60.57,60.57,34.17,77.58,30,0,4.29,IN,1598812944,1264976,
427,Bloemfontein,39.20,39.20,-29.12,26.21,25,0,3.36,ZA,1598812980,1018725,


In [8]:
# find the closest hotel to coordinates
#Set the parameters
params = {
    "key": gkey,
    "radius": 5000,
    "types": "lodging"
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #get latiude and longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    #add the locations to the parameters of the url
    params["location"] = f"{str(lat)},{str(lng)}"
    #print(params)    
    
    #Use the following url for search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # assemble url and make API request
    response = requests.get(base_url, params=params)
    #print(response.url)
    response = response.json()
    #print(requests.get(base_url, params=params))
    print(json.dumps(response, indent= 4, sort_keys= True))    
    
    # extract results
    results = response["results"]
    #print(response.url)
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        
    except:
        pass


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -14.8219765,
                    "lng": 34.6358702
                },
                "viewport": {
                    "northeast": {
                        "lat": -14.8207317697085,
                        "lng": 34.6369740302915
                    },
                    "southwest": {
                        "lat": -14.8234297302915,
                        "lng": 34.63427606970851
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Dear Executive Lodge",
            "photos": [
                {
                    "height": 768,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111270646226098636647\">flexor deeejay sopra

/Users/dianaestrada/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 49.70424260000001,
                    "lng": 37.1802184
                },
                "viewport": {
                    "northeast": {
                        "lat": 49.7056976302915,
                        "lng": 37.1812877802915
                    },
                    "southwest": {
                        "lat": 49.7029996697085,
                        "lng": 37.1785898197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Hotel\u02b9 Komunal\u02b9noho Pidpryyemstva Akva",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 3968,
                    "html_attributions": [
     

{
    "html_attributions": [],
    "next_page_token": "CqQCHwEAAIjjfPyx9XBovjewqk1UIozQ481-pnFnRM9hc_-Lkn8DIaNXX_zAwmbx1Tdiu2fMbfHm6BSgrRj2dAd9ODd4eMAlqgBRv6dC3q2NANxW_f6ezGUKTz9ucnMWCj-kTq5Dkl9CB0bQ2ok8GMmCMduWX5naGDTUFIwixjMbvbjf335tcwpUysMWzhRv-9WpOLCyb3JRWgtx5Dvvq23y2fvYvxAYsMUqjSW9ehoHdp-imGQwdpccpW6ulm0KrxQA19TqsCEKRk9VAXn1SsNLm-Jr563fn4S-3mouXnSCjqJLF8fzbiXWKxeoN_6CMYtP_kttvIKUYiyV-HCZ2cVomm3Y4Pqm8jKRvKAizqr7-BTUCTm0E-I_XV1Qr-_qPfGODC-hxRIQ6AKmJg3nDyI3JAcYkxfgcBoUgJ37ASxtdWqscI43OFM9bbWRgcA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -31.53712509999999,
                    "lng": -68.5239433
                },
                "viewport": {
                    "northeast": {
                        "lat": -31.5357914697085,
                        "lng": -68.52259271970848
                    },
                    "southwest": {
                        "lat": -31.538

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.065264,
                    "lng": 15.7250709
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.0639306697085,
                        "lng": 15.7263528802915
                    },
                    "southwest": {
                        "lat": -17.0666286302915,
                        "lng": 15.7236549197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "SOS Motel",
            "place_id": "ChIJ5xR9v41glBsRsr2CeK_WmII",
            "plus_code": {
                "compound_code": "WPMG+V2 Ondjiva, Angola",
                "global_co

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.73679670000001,
                    "lng": 69.5265893
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.7381345302915,
                        "lng": 69.5280925302915
                    },
                    "southwest": {
                        "lat": 36.7354365697085,
                        "lng": 69.5253945697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "\u0627\u06cc\u0633\u062a\u06af\u0627\u0647 \u062e\u0648\u0627\u062c\u0647 \u063a\u0627\u0631",
            "place_id": "ChIJCcSZwKfRzjgRsRynucK0Wko",
            "plus_code": {
                "compound_code": "PGPG+PJ Taleqan, Afghanistan",
              

{
    "html_attributions": [],
    "next_page_token": "CqQCHwEAAN1IwEv_Ydf9quWdSxLH_UmZ_XRRM8kR_zZDfkZ5ip1ZKBgK41ZTfZX0zSCNIT4cgVo7-yxQiya18GjqqlPQlPSw37Cj7nrtYq7q5JUOvUEfWgl_ddYfhED1EwxQDT8Wf5wWdAlD7X5xK7LoEyiO2Ox2i2UUBdLi6bq8-wckIaKoGNF_s57uTHqHsMEJqIByMrykKko0uZFcgPwZe4msZYneXaUkldToyvLtUh0aXzn_zM9GENfNupVQ3N0RJkvTYwdFm6qkWjipiP0STs-DewndD4I-biRhL9eb4Q1jGJSbBQuRgr35TPtAQ2W3I9hzMmGwOZWsS5fEX6o3rwKz1AMwa7A7nFeCAC6RGi61h7j0pGtJ_ELA73pxBsVEiId8xxIQPY5yP4NDpMkA3l6zEV_mbhoUVcPQWeqy5ZwTvXlEx8WjdrOgbJk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 34.16175459999999,
                    "lng": 77.57566829999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.1630703302915,
                        "lng": 77.5771185802915
                    },
                    "southwest": {
                        "lat": 34.16

{
    "html_attributions": [],
    "next_page_token": "CqQCHwEAAK01G30pKvMrWigqnwoQA8A2E6bTI64ex6BmZ7tbQdfhXcgl5J1seyMLxCJBmjsI3_pY-BR0wfzJpFaBzjvkia6kQEwayMtM3pf37a8OYky-QLvMEbrDHlXcHm9wskdSFeb8eDwrNvaxwAlFdTIDx52irXX-xDRZt5IlY_rYCxUC1p7TAj1L61nNwY5Ad6h9SJIKx684TUjiMb-y7PFuHEWXn_rzv9jYhh-plzrE_4U97BFNYbh0vpGHp99MZ_hGff6XzpI1LAP8aivUX42Jycuw9Ph2NaOLDAnaaN_AW5SifIZ5ynsd8JmhyYyQvCL_g32G1ArBQM_1GD6cgWJQHCNiFv4oqxy3GWdjq0MUCio6A-Ixj6o0GHjm8_984MqBWxIQ0h3dy0zZOjP14xL8W2CrzBoUZ7kpRISDSXyYuDjuuQoj5g4wES4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.10557309999999,
                    "lng": 26.1918112
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.1043963197085,
                        "lng": 26.1930840802915
                    },
                    "southwest": {
                        "lat": -29.107094

In [9]:
hotel_df

,City,Temperature F,Max Temperature F,Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Country,Date,City Id,Hotel Name
56,Ntcheu,69.42,69.42,-14.82,34.64,33,0,3.74,MW,1598812778,924492,Dear Executive Lodge
121,Shevchenkove,75.11,75.20,49.70,37.17,33,0,4.47,UA,1598812804,694137,Hotelʹ Komunalʹnoho Pidpryyemstva Akva
124,Menongue,65.62,65.62,-14.66,17.69,32,6,2.37,AO,1598812805,3347353,Ritz Lauca
182,San Juan,60.78,60.78,-31.54,-68.54,32,0,3.80,AR,1598812668,3837213,Hotel Provincial
228,Yumen,55.74,55.74,40.28,97.20,38,0,4.27,CN,1598812865,1528998,
247,Ondjiva,70.57,70.57,-17.07,15.73,25,1,4.12,AO,1598812877,3346821,SOS Motel
358,Taloqan,65.57,65.57,36.74,69.53,36,3,2.39,AF,1598812941,1123004,ایستگاه خواجه غار
362,Leh,60.57,60.57,34.17,77.58,30,0,4.29,IN,1598812944,1264976,Hotel Ladakh Greens
427,Bloemfontein,39.20,39.20,-29.12,26.21,25,0,3.36,ZA,1598812980,1018725,Protea Hotel by Marriott Bloemfontein


In [10]:
# Convert Hotel Name as a list
hotel_name = hotel_df["Hotel Name"].tolist()
city_name = hotel_df["City"].tolist()
country_name = hotel_df["Country"].tolist()

# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Latitude', 'Longitude']]

# Create a marker_layer using the hotel_name list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {hotel}, City: {city}, Country: {country}" for hotel, city, country in zip(hotel_name, city_name, country_name)])
fig.add_layer(markers)
fig

#--------------------
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Store humidity in weight
weights = weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))